<a href="https://colab.research.google.com/github/PintoBI/Cole-Moore/blob/main/Cole_Moore_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/pintobi/Cole-Moore

Cloning into 'Cole-Moore'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 1.27 MiB | 8.13 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
# Import the module
import sys
sys.path.append('/content/Cole-Moore')
from data_loader import load_data

# Import other necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from google.colab import files
import os

# Upload a data file
print("Please upload your electrophysiology data file (e.g., .csv, .txt, .asc). If not file is ulpoaded example simulated data will be used")
uploaded = files.upload()

# Process the uploaded file
if uploaded:
    file_name = next(iter(uploaded))
    file_content = uploaded[file_name]
    print(f"\nUploaded file: {file_name}")
elif os.path.exists('/content/Cole-Moore/Example/simulation.txt'):
    file_name = '/content/Cole-Moore/Example/simulation.txt'
    with open(file_name, 'rb') as f:
        file_content = f.read()
    print(f"\nNo file uploaded. Using sample file: {file_name}")
else:
    print("No file uploaded and sample file not found. Please upload a file manually.")
    file_name = None
    file_content = None

if file_name and file_content:
    # Use your load_data function
    data = load_data(
        filename=file_name,
        file_content=file_content,
        header_lines=1,
        time_col_hint='Time',
        delimiter=None
    )

    if data is not None:
        print("\n--- Data Preview ---")
        display(data.head())


Please upload your electrophysiology data file (e.g., .csv, .txt, .asc). If not file is ulpoaded example simulated data will be used


In [ ]:
"""
Example script for using the voltage_converter module

This script assumes that 'data' has already been loaded and is available in the environment.
It converts sweep column headers to voltage values using the voltage_converter module.
"""

# Import the voltage_converter module
from voltage_converter import rename_sweep_columns_to_voltage

# Check if data is available
if 'data' in locals() and data is not None:
    print("\n--- Renaming Sweep Columns to Voltage Values ---")

    # User-specified voltage parameters
    initial_voltage = -60   # mV, voltage of the first sweep
    voltage_step = -5     # mV, voltage difference between consecutive sweeps

    print(f"Initial voltage: {initial_voltage} mV")
    print(f"Voltage step: {voltage_step} mV")

    # Rename the columns
    voltage_data = rename_sweep_columns_to_voltage(
        data,
        time_col='Time (ms)',
        initial_voltage=initial_voltage,
        voltage_step=voltage_step
    )

    # Replace data with the renamed version for subsequent processing
    data = voltage_data

    # Display the first few rows to verify
    print("\n--- Data with Voltage Headers (first 5 rows) ---")
    from IPython.display import display
    display(data.head())

    print("\n--- Column Renaming Complete ---")

else:
    print("\n--- Skipping Sweep to Voltage Renaming ---")
    print("Variable 'data' not found. Please run the data import cell first.")

In [ ]:
"""
Example script for using the normalizer module

This script assumes that 'data' has already been loaded and is available in the environment.
It normalizes the data using the specified method and time window.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Import the normalizer module
from normalize import normalize_data_window

# --- Configure and Apply Normalization ---
# Check if data exists
if 'data' in locals() and data is not None:
    print("\n--- Configuring Normalization ---")

    # --- Select Normalization Parameters ---
    # Options: 'subtract', 'divide', 'normalize', 'none'
    normalization_method = 'divide'

    # Time window (in ms) for calculating baseline or min/max
    # Set to None to use the start/end of the data trace
    norm_start_time_ms = 1     # ms, Start time for normalization window
    norm_end_time_ms = 100      # ms, End time for normalization window

    print(f"Selected Method: {normalization_method}")
    print(f"Calculation Window: [{norm_start_time_ms}, {norm_end_time_ms}] ms")

    # Apply normalization
    normalized_data = normalize_data_window(
        data,
        time_col='Time (ms)',  # Should be standard name from load_data
        method=normalization_method,
        start_time_ms=norm_start_time_ms,
        end_time_ms=norm_end_time_ms
    )

    if normalized_data is not None:
        print("\n--- Normalized Data (first 5 rows) ---")
        display(normalized_data.head())

        # Optional: Basic plot to compare raw vs normalized for the first sweep
        first_sweep = normalized_data.columns[4]  # Get name of first sweep column
        plt.figure(figsize=(10, 4))
        plt.plot(data['Time (ms)'], data[first_sweep], label=f'{first_sweep} (Raw)', alpha=0.7)
        plt.plot(normalized_data['Time (ms)'], normalized_data[first_sweep], label=f'{first_sweep} (Normalized)', alpha=0.7)
        plt.title(f'Raw vs Normalized Data ({normalization_method}) - {first_sweep}')
        plt.xlabel('Time (ms)')
        plt.ylabel('Current')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.show()

    else:
        print("\n--- Normalization Failed ---")
        # You might want to assign data to normalized_data here if you want subsequent cells to run
        # normalized_data = data.copy()
        # print("Using raw data for subsequent steps as normalization failed.")

else:
    print("\n--- Skipping Normalization ---")
    print("Variable 'data' not found. Please run the data import cell first.")
    normalized_data = None  # Ensure variable exists but is None


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from google.colab import files

from exponential_fit import analyze_with_exponential_fit, visualize_exponential_fits
# (Optionally available too: analyze_single, analyze_double, analyze_triple)

# --- Run Exponential Fit Analysis ---
if 'normalized_data' in locals() and normalized_data is not None:
    print("\n--- Configuring Exponential Fit Analysis ---")

    # --- Set Analysis Parameters ---
    exp_type = 'single'        # 'single' | 'double' | 'triple'
    # n_exp = 3                 # (optional override) 1, 2, or 3 — uncomment to force it

    start_time = 0.6          # ms
    end_time = 100              # ms
    baseline_percent = 30      # %
    max_percent = 99           # %
    display_max_time = 10      # ms

    print(f"Selected Exponential Type: {exp_type}")
    print(f"Fitting Window: [{start_time}, {end_time if end_time is not None else 'end'}] ms")
    print(f"Fitting Range: {baseline_percent}% to {max_percent}% of amplitude")
    print(f"Display Range: 0 to {display_max_time} ms")

    # Run the analysis
    exp_fit_results = analyze_with_exponential_fit(
        normalized_data,
        time_col='Time (ms)',
        start_time=start_time,
        end_time=end_time,
        baseline_percent=baseline_percent,
        max_percent=max_percent,
        exp_type=exp_type,   # keep this...
        # n_exp=n_exp,       # ...or pass this to override exp_type
    )

    # Visualize (NOTE: no exp_type parameter here)
    if exp_fit_results:
        results_df = visualize_exponential_fits(
            normalized_data,
            exp_fit_results,
            time_col='Time (ms)',
            x_min=0,
            x_max=display_max_time,
            max_sweeps=6
        )

        # Save & download
        if results_df is not None:
            results_filename = f"cole_moore_exp_fit_results_{exp_type}.csv"
            results_df.to_csv(results_filename, index=False)
            print(f"Results saved to {results_filename}")
            #files.download(results_filename)

else:
    print("\n--- Skipping Exponential Fit Analysis ---")
    print("Variable 'normalized_data' not found. Please run the data normalization cell first.")


In [ ]:
"""
Script for using the trace_aligner module for Cole-Moore analysis

This script aligns traces by their rising phase (activation kinetics) and visualizes
the results. It handles both positive and negative currents automatically.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from google.colab import files

# Import the trace_aligner module
from trace_aligner import align_traces, visualize_aligned_traces, plot_cole_moore_shifts

# --- Run Trace Alignment Analysis ---
if 'normalized_data' in locals() and normalized_data is not None:
    print("\n--- Configuring Trace Alignment Analysis ---")

    # --- Set Analysis Parameters ---
    rising_phase_only = True       # Focus only on rising phase (activation) for alignment
    alignment_method = 'single_threshold'  # 'single_threshold', 'multi_threshold', or 'curve_fit'
    alignment_threshold = 0.5      # For single threshold method - percent of amplitude (0.0-1.0)
    threshold_points = 1           # For multi-threshold method - number of points to use
    exclude_outliers = False        # Exclude outliers from alignment
    reference_sweep = '-60 mV'     # None for automatic selection based on typical kinetics
    start_time = 0.1               # ms, Start time for analysis (None for all data)
    max_shift = 50.0               # ms, Maximum allowable shift
    display_x_max = 15             # ms, Maximum time to display on x-axis
    auto_detect_polarity = True    # Automatically detect negative currents
    num_sweeps = 6

    print(f"Alignment Method: {alignment_method}")
    if alignment_method == 'single_threshold':
        print(f"Alignment Threshold: {alignment_threshold*100}% of peak")
    elif alignment_method == 'multi_threshold':
        print(f"Using {threshold_points} threshold points for robust alignment")

    print(f"Reference Sweep: {'Auto-select' if reference_sweep is None else reference_sweep}")
    print(f"Analysis Start Time: {start_time if start_time is not None else 'Beginning of trace'} ms")
    print(f"Exclude Outliers: {'Yes' if exclude_outliers else 'No'}")
    print(f"Rising Phase Only: {'Yes' if rising_phase_only else 'No'}")
    print(f"Auto-Detect Polarity: {'Yes' if auto_detect_polarity else 'No'}")

        # Run the alignment
    shift_results = align_traces(
        normalized_data,
        time_col='Time (ms)',
        reference_sweep=reference_sweep,
        start_time=start_time,
        rising_phase_only=rising_phase_only,
        alignment_threshold=alignment_threshold,
        alignment_method=alignment_method,
        threshold_points=threshold_points,
        exclude_outliers=exclude_outliers,
        max_shift=max_shift,
        auto_detect_polarity=auto_detect_polarity

    )

    # Visualize the results
    if shift_results:
        results_df = visualize_aligned_traces(
            normalized_data,
            shift_results,
            time_col='Time (ms)',
            max_sweeps=num_sweeps ,  # Maximum number of sweeps to show
            x_min=0,
            x_max=display_x_max,
            highlight_rising_phase=False,
            auto_detect_polarity=auto_detect_polarity
        )

        # Create a separate plot of Cole-Moore shift vs. voltage
        plot_cole_moore_shifts(results_df)

        # Save results to CSV
        if results_df is not None:
            results_filename = "cole_moore_shift_results.csv"
            results_df.to_csv(results_filename, index=False)
            print(f"Results saved to {results_filename}")

            # Download file in Colab (uncomment to enable)
            # files.download(results_filename)
    else:
        print("No valid alignment results to visualize.")

else:
    print("\n--- Skipping Trace Alignment Analysis ---")
    print("Variable 'normalized_data' not found. Please run the data normalization cell first.")

In [ ]:
"""
Script for using the derivative_analysis module for Cole-Moore analysis

This script assumes that 'raw_data' has already been loaded and is available
in the environment. It performs derivative peak analysis and visualizes the results.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from google.colab import files

# Import the derivative_analyzer module
from derivative_analysis import analyze_derivative_peaks, visualize_derivative_peaks

# --- Run Derivative Peak Analysis ---
if 'data' in locals() and data is not None:
    print("\n--- Configuring Derivative Peak Analysis ---")

    # --- Set Analysis Parameters ---
    start_time = 0.1         # ms, Start time for analysis (None to use all data)
    window_length = 11       # Window length for smoothing (must be odd)
    polyorder = 3           # Polynomial order for Savitzky-Golay filter
    min_peak_height = None     # Minimum height for peak detection (None for auto)
    peak_prominence = 1e-7   # Minimum prominence for peak detection
    num_sweeps = 6             # Num of sweeps to plot

    # Display parameters
    x_min = 0                  # ms, Minimum time for x-axis in plots
    x_max = 10                 # ms, Maximum time for x-axis in plots
    y_min_deriv = -2          # Minimum value for y-axis in derivative plot (None for auto)
    y_max_deriv =20 # Maximum value for y-axis in derivative plot (None for auto)

    print(f"Analysis Start Time: {start_time} ms")
    print(f"Smoothing Window: {window_length} points")
    print(f"Polynomial Order: {polyorder}")
    print(f"Plot Range X: {x_min} to {x_max} ms")
    print(f"Plot Range Y for derivative: {y_min_deriv} to {y_max_deriv} (auto if None)")

    # Run the analysis on data
    print("\nAnalyzing data with derivative peak method...")
    derivative_results = analyze_derivative_peaks(
        data,
        time_col='Time (ms)',
        start_time=start_time,
        window_length=window_length,
        polyorder=polyorder,
        min_peak_height=min_peak_height,
        peak_prominence=peak_prominence,
        early_activation_bias= False
    )

    # Visualize the results
    if derivative_results:
        results_df = visualize_derivative_peaks(
            data,
            derivative_results,
            time_col='Time (ms)',
            window_length=window_length,
            polyorder=polyorder,
            start_time=start_time,
            x_min=x_min,
            x_max=x_max,
            max_sweeps=num_sweeps,
            y_min_deriv=y_min_deriv,
            y_max_deriv=y_max_deriv
        )

        # Save results to CSV
        if results_df is not None:
            results_filename = "cole_moore_derivative_peak_results.csv"
            results_df.to_csv(results_filename, index=False)
            print(f"Results saved to {results_filename}")

else:
    print("\n--- Skipping Derivative Peak Analysis ---")
    print("Variable 'data' not found. Please run the data import cell first.")

In [ ]:
"""
Script for using the cole_moore_comparator module

This script assumes that the analysis result CSV files exist in the current directory.
It compares the Cole-Moore shift results from different analysis methods.
"""

# Import the cole_moore_comparator module
from compare_methods import compare_cole_moore_methods

# --- Run Cole-Moore Methods Comparison ---
print("\n--- Comparing Cole-Moore Analysis Methods ---")

# Specify paths to the CSV files from each method
exp_file = "cole_moore_exp_fit_results_single.csv"
shift_file = "cole_moore_shift_results.csv"
derivative_file = "cole_moore_derivative_peak_results.csv"

# Check for existing files
import os
all_files_exist = True

for file_path, method_name in [
    (exp_file, "Exponential Fit"),
    (shift_file, "Trace Alignment"),
    (derivative_file, "Derivative Peak")
]:
    if not os.path.exists(file_path):
        print(f"Warning: {file_path} not found. {method_name} results will not be included.")
        all_files_exist = False

if all_files_exist:
    print("All result files found. Proceeding with comparison.")
else:
    print("Note: Missing files will be skipped in the comparison.")

# Run comparison
compare_cole_moore_methods(
    exp_csv=exp_file,
    shift_csv=shift_file,
    derivative_csv=derivative_file,
    invert_shifts=True
)

print("\n--- Comparison Complete ---")


In [ ]:
from cole_moore_correlation import correlate_cole_moore

correlate_cole_moore(
    exp_csv="cole_moore_exp_fit_results_single.csv",
    shift_csv="cole_moore_shift_results.csv",
    derivative_csv="cole_moore_derivative_peak_results.csv",
    invert_shifts=True
)

In [ ]:
from cole_moore_baseline_correlation import correlate_cole_moore_baselined

# Usage example for Google Colab:
print("Cole-Moore Methods Correlation Analysis with Baselined Data")
print("=" * 60)

# Specify your CSV file paths here
exp_file = "cole_moore_exp_fit_results_single.csv"
shift_file = "cole_moore_shift_results.csv"
derivative_file = "cole_moore_derivative_peak_results.csv"

# Run the correlation analysis with baselined data
correlate_cole_moore_baselined(
    exp_csv=exp_file,
    shift_csv=shift_file,
    derivative_csv=derivative_file,
    invert_shifts=True  # Set to False if you don't want to invert trace alignment shifts
)